In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import json
from pathlib import Path

import numpy as np

import tensorflow as tf

from phagenet import PredictionGenerator, DataGenerator
from phagenet.preprocessing import preprocess_assemblies, preprocess_assemblies_gcp
from phagenet.models import LSTMClassifier

In [3]:
input_dir = Path.cwd() / '..' / 'input'
input_dir.mkdir(exist_ok=True)

output_dir = Path.cwd() / '..' / 'output'
output_dir.mkdir(exist_ok=True)

weights_dir = Path.cwd() / '..' / 'weights'
weights_dir.mkdir(exist_ok=True)

data_dir = Path.cwd() / '..' / '..' / 'prophage-tool' / 'out' / 'phageboost_output'

In [4]:
k = 7
model_name = 'lstm-64-transfer'

classifier = LSTMClassifier(k, 64)
classifier.set_checkpoint_path(weights_dir / 'lstm-64')
classifier.load(40)
classifier.set_checkpoint_path(weights_dir / model_name)
classifier.model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
phage-7-mer-freq (InputLayer)   [(None, 16384)]      0                                            
__________________________________________________________________________________________________
bacteria-7-mer-freq (InputLayer [(None, 16384)]      0                                            
__________________________________________________________________________________________________
reshape (Reshape)               (None, 1, 16384)     0           phage-7-mer-freq[0][0]           
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 1, 16384)     0           bacteria-7-mer-freq[0][0]        
______________________________________________________________________________________________

In [5]:
prophage_labels = {}
prophage_interactions = []
prophage_assemblies = {}

for t in os.scandir(data_dir):
    for a in os.scandir(t.path):
        bact_assembly = a.name
        for f in os.scandir(a.path):
            if f.name.endswith('.fasta'):
                phage_assembly = f.name.replace('.fasta', '')
                prophage_assemblies[phage_assembly] = f.path
                inter = f"{phage_assembly},{bact_assembly}"
                prophage_interactions.append(inter)
                prophage_labels[inter] = 1

In [6]:
prophages, _ = preprocess_assemblies(k, prophage_assemblies, out_dir=input_dir, out_prefix='prophages.capstone')

KeyboardInterrupt: 

In [6]:
np.random.shuffle(prophage_interactions)
ptest_set = prophage_interactions[:100]
ptrain_set = prophage_interactions[100:]

In [7]:
with open(input_dir / 'interactions.final.json') as f:
    interactions = np.array(json.load(f))

class_sizes = (
    len([i for i in interactions if i[2] == '1']),
    len([i for i in interactions if i[2] == '0'])
)
test_sample_split = (1000, 9000)

test_idxs = np.concatenate([
    np.random.choice(class_sizes[0], size=test_sample_split[0], replace=False),
    np.random.choice(class_sizes[1], size=test_sample_split[1], replace=False) + class_sizes[0]
])

labels, train_interactions, test_interactions = {}, [], []
for i, interaction in enumerate(interactions):
    interaction_str = "{0},{1}".format(interaction[0], interaction[1])
    labels[interaction_str] = int(interaction[2])
    if i in test_idxs:
        test_interactions.append(interaction_str)
    else:
        train_interactions.append(interaction_str)

labels.update(prophage_labels)
train_interactions = np.concatenate([train_interactions, ptrain_set])
test_interactions = np.concatenate([test_interactions, ptest_set])

In [8]:
train_generator = DataGenerator(k, train_interactions, labels, 
    [(input_dir / 'phages.ids.npy'), (input_dir / 'prophages.capstone.ids.npy')], 
    [(input_dir / 'phages.kmer-freqs.npy'), (input_dir / 'prophages.capstone.kmer-freqs.npy')], 
    [(input_dir / 'bact.ids.npy'), (input_dir / 'bact.capstone.ids.npy')], 
    [(input_dir / 'bact.kmer-freqs.npy'), (input_dir / 'bact.capstone.kmer-freqs.npy')],
    batch_size=128)

test_generator = DataGenerator(k, test_interactions, labels, 
    [(input_dir / 'phages.ids.npy'), (input_dir / 'prophages.capstone.ids.npy')], 
    [(input_dir / 'phages.kmer-freqs.npy'), (input_dir / 'prophages.capstone.kmer-freqs.npy')], 
    [(input_dir / 'bact.ids.npy'), (input_dir / 'bact.capstone.ids.npy')], 
    [(input_dir / 'bact.kmer-freqs.npy'), (input_dir / 'bact.capstone.kmer-freqs.npy')],
    batch_size=len(test_interactions))

ptest_generator = DataGenerator(k, ptest_set, labels, (input_dir / 'prophages.capstone.ids.npy'), (input_dir / 'prophages.capstone.kmer-freqs.npy'), (input_dir / 'bact.capstone.ids.npy'), (input_dir / 'bact.capstone.kmer-freqs.npy'), batch_size=len(ptest_set))


In [9]:
classifier.compile()

In [13]:
classifier.fit(20, train_generator, log_dir=(output_dir / "logs" / model_name), validation_data=test_generator[0])

Epoch 41/60
1491/1491 [==============================] - 67s 41ms/step - loss: 0.0270 - tp: 5965.0000 - fp: 1545.0000 - tn: 183297.0000 - fn: 41.0000 - accuracy: 0.9917 - precision: 0.7943 - recall: 0.9932 - auc: 0.9991 - val_loss: 0.0181 - val_tp: 1093.0000 - val_fp: 49.0000 - val_tn: 8951.0000 - val_fn: 7.0000 - val_accuracy: 0.9945 - val_precision: 0.9571 - val_recall: 0.9936 - val_auc: 0.9996
Epoch 42/60
1491/1491 [==============================] - 59s 40ms/step - loss: 0.0158 - tp: 5984.0000 - fp: 913.0000 - tn: 183929.0000 - fn: 22.0000 - accuracy: 0.9951 - precision: 0.8676 - recall: 0.9963 - auc: 0.9996 - val_loss: 0.0189 - val_tp: 1084.0000 - val_fp: 54.0000 - val_tn: 8946.0000 - val_fn: 16.0000 - val_accuracy: 0.9931 - val_precision: 0.9525 - val_recall: 0.9855 - val_auc: 0.9996
Epoch 43/60
1491/1491 [==============================] - 59s 39ms/step - loss: 0.0107 - tp: 5996.0000 - fp: 702.0000 - tn: 184140.0000 - fn: 10.0000 - accuracy: 0.9963 - precision: 0.8952 - recall: 0.